In [66]:
import MySQLdb

import traceback
import sys
import os
path = 'C:/Users/Hsiao/Python/files_local_backup/text_files'
try:
    db = MySQLdb.connect("localhost", "root", "root", "foresee_project")
    cursor = db.cursor()
    i = 0
    for dv in os.listdir(path):
        for fname in os.listdir(os.path.join(path, dv))[1:10]:
            fpath = os.path.join(path, dv, fname)
            fo = open(fpath , 'r')
            fc = fo.read()
            fo.close()

            newtext = ''.join(fc.split())
            newtext = ''.join(newtext.split("　"))
            newtext = ''.join(newtext.split(","))

            totalDamage = getTotalDamage(newtext)
            totalFee = getJudgeFee(newtext, totalDamage)

            dic = {'court':fname[0:4].decode('big5'),
                   'judgeResult':getJudgeResult(newtext),
                   'accidentDate':getAccidentDate(newtext),
                   'judgeDate': getJudgeDate(newtext),
                   'assertionExisting':getAssertionExisting(newtext),
                   'judge': getJudge(fc, newtext),
                   'judgeFee':totalFee,
                   'judgeReason':getJudgeReason(newtext),
                   'judgeId':fname[5:-4].decode('big5'),
                   'totalDamage':totalDamage,
                   'pFee_dFee':getPDFee(newtext, totalFee)
                  }
            print str(i), dic['court'], dic['accidentDate'], dic['judgeDate'], dic['assertionExisting'], \
                    dic['judge'], dic['judgeFee'], dic['judgeReason'], dic['judgeId'], dic['totalDamage'], \
                    dic['pFee_dFee']
            sql = "INSERT INTO judgement(ID, Court, AccidentDate, JudgeDate, AssertionExisting, Judge, \
                   JudgeFee, JudgeReason, JudgeId, TotalDamage, PFee_DFee) \
                   VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%d', '%s', '%s','%d', '%s')" % \
                   (str(i), dic['court'].encode('utf-8'), dic['accidentDate'].encode('utf-8'), \
                    dic['judgeDate'].encode('utf-8'), dic['assertionExisting'].encode('utf-8'), \
                    'judge', dic['judgeFee'], dic['judgeReason'], dic['judgeId'].encode('utf-8'), \
                    dic['totalDamage'], dic['pFee_dFee'].encode('utf-8') )
            try:
                cursor.execute(sql)
            except:
                continue
            
            i += 1
    db.commit()

finally:
    db.close()

0 南投 99, 8, 16 1000622 True, True 鍾淑慧 1000 損害賠償 100_埔小_2 53098 0, 1000
1 南投 100, 5, 29 1000801 True, True 趙思芸 1000 損害賠償 100_埔小_81 47053 529, 471
2 南投 99, 4, 28 1010425 True, True 鍾淑慧 1000 損害賠償 100_埔簡_113 21120 580, 420
3 南投 99, 2, 11 1001226 True, True 趙思芸 2100 損害賠償 100_埔簡_131 73945 1292, 808
4 南投  1020717 True, True 鍾淑慧 3090 損害賠償 100_埔簡_153 158700 1430, 1660
5 南投 98, 8, 28 1000607 True, True 趙思芸 1440 侵權行為損害賠償 100_埔簡_22 22891 1188, 252
6 南投 98, 7, 20 1000718 True, True 趙思芸 4300 損害賠償 100_埔簡_47 0 3240, 1060
7 南投  1010216 True, True 鍾淑慧 1000 損害賠償 100_投勞簡_7 0 1000, 0
8 南投 100, 1, 6 1000428 True, True 鍾淑慧 1000 損害賠償 100_投小_104 25485 0, 1000
9 嘉義  1000425 True, True 蕭奕弘 1000 侵權行為損害賠償 100_嘉小_143 0 1000, 0
10 嘉義 99, 11, 3 1000425 True, True 　蕭奕弘 1000 侵權行為損害賠償 100_嘉小_144 40646 0, 1000
11 嘉義 99, 7, 23 1000210 True, True 林雅莉 1000 侵權行為損害賠償 100_嘉小_16 30800 0, 1000
12 嘉義 98, 4, 22 1000429 True, True 林雅莉 1000 侵權行為損害賠償 100_嘉小_162 37795 550, 450
13 嘉義 99, 6, 26 1000707 True, True 蕭奕弘 1000 侵權行為損害賠償 100_嘉

### Functions

In [20]:
# separate judgeFee
def getPDFee(txt, totalJudgeFee):
    try:
        import re
        total = totalJudgeFee
#         block = txt.split('主張')[0].split('】')[-1]
#         terms = ['事實及理由', '事實', '程序']
#         for term in terms:
#             if term in block:
#                 nblock = block.split(term)[-2]
#                 break;
        endterm = '【裁判全文】'
        if '事實及理由' in txt:
            block = txt.split('事實及理由')[0].split(endterm)[-1]
        elif '理由要領' in txt:
            block = txt.split('理由要領')[0].split(endterm)[-1]
        else:
            block = txt.split('主張')[0].split(endterm)[-1]
        delimiter = u"，|。|：|；|（|）" 
        aryPlaintiff = []
        aryDefendant = []
        (PFee, DFee) = (0, 0)
        for ln in re.split(delimiter, ''.join(block.decode('utf-8'))):
            for i in range(0, ln.count(u'、')+1):
                if u'原告' in ln and u'負擔' in ln:
                    if u'元' in ln:
                        if u'幣' in ln:
                            aryPlaintiff.append(transform(ln.split(u'幣')[-1].split(u'元')[0]))
                        else:
                            aryPlaintiff.append(transform(ln.split(u'負擔')[-1].split(u'元')[0]))
                    elif u'分之' in ln:
                        aryPlaintiff.append(fractions(ln.split(u'負擔')[1])*total)
                    elif u'餘' in ln:
                        aryPlaintiff.append(total - DFee - PFee)
                    elif u'由原告' in ln:
                        aryPlaintiff.append(total)
                    PFee = int(sum(aryPlaintiff))
                if u'被告' in ln and u'負擔' in ln:
                    if u'元' in ln:
                        if u'幣' in ln:
                            aryDefendant.append(transform(ln.split(u'幣')[-1].split(u'元')[0]))
                        else:
                            aryDefendant.append(transform(ln.split(u'負擔')[-1].split(u'元')[0]))
                    elif u'分之' in ln:
                        aryDefendant.append(fractions(ln.split(u'負擔')[1])*total)
                    elif u'餘' in ln:
                        aryDefendant.append(total - PFee - DFee)
                    elif u'由被告' in ln:
                        aryDefendant.append(total)
                    DFee = int(sum(aryDefendant))

#         return [PFee, DFee]
        return str([PFee, DFee]).strip('[]')
    except:
#         return 'error'
        return None
    
# total judgeFee
import re
def getJudgeFee(txt, totalDamage):
    ary = []
    damage = totalDamage

    if damage != 'error':
        ary.append(int(pyjs.call('calculate', damage)))
    else:
        ary.append(None)
    
    delimiter = u"，|。|、"  
    try:
        if '計算書' in txt and '計' in txt:
            total = txt.split('法官')[1].split('計算書')[1].split('計')[1].split('元')[0]
        else:
            total = ''
        if total is not None and total.isdigit():
            total = int(total)
            ary.append(int(total))
        else:
            for i in re.split(delimiter, ''.join(txt.decode('utf-8'))):
                if u'訴訟費用'in i and u'元' in i:
                    total = transform(i.split(u'幣')[1].split(u'元')[0])
                    ary.append(int(total))
                    break
    except:
        ary.append(None)
        
    # ---------------------------    
    endterm = '【裁判全文】'
    if '事實及理由' in txt:
        block = txt.split('事實及理由')[0].split(endterm)[-1]
    elif '理由要領' in txt:
        block = txt.split('理由要領')[0].split(endterm)[-1]
    else:
        block = txt.split('主張')[0].split(endterm)[-1]
    delimiter = u"，|。|：|；|（|）" 
    aryPlaintiff = []
    aryDefendant = []
    (PFee, DFee) = (0, 0)
    for ln in re.split(delimiter, ''.join(block.decode('utf-8'))):
        for i in range(0, ln.count(u'、')+1):
            if u'原告' in ln and u'負擔' in ln and u'元' in ln:
                if u'幣' in ln:
                    aryPlaintiff.append(transform(ln.split(u'幣')[-1].split(u'元')[0]))
                else:
                    aryPlaintiff.append(transform(ln.split(u'負擔')[-1].split(u'元')[0]))
            PFee = int(sum(aryPlaintiff))
            if u'被告' in ln and u'負擔' in ln and u'元' in ln:
                if u'幣' in ln:
                    aryDefendant.append(transform(ln.split(u'幣')[-1].split(u'元')[0]))
                else:
                    aryDefendant.append(transform(ln.split(u'負擔')[-1].split(u'元')[0]))
            DFee = int(sum(aryDefendant))
    if DFee != 0 and PFee != 0:
        ary = [sum([PFee, DFee])]
# ---------------------------
    return max(ary)

# Judge
import jieba
import re
def getJudge(txt1, txt2):
    if '書記官' in txt1:
        term = '書記官'
    if '書  記  官' in txt1:
        term = '書  記  官'
    try:
        judge = txt1.split(term)[-2].split('官')[-1].split()[0]
        if len(judge) > 3 and '【裁判字號】' not in judge:
            return judge
        else:
            text = txt2.split('書記官')[-2].split('法官')[-1]
            jieba.load_userdict('userdict.txt')
            words = jieba.cut(text, cut_all=False)
            return words.next()
    except:
#         return 'error'
        return None
    
# Judge Date
def getJudgeDate(txt):
    date = txt.split('【裁判日期】')[1].split('【裁判案由】')[0]
#     return [date[0:-4], date[-4:-2], date[-2:]]
    return '%s%s%s' % (date[0:-4], date[-4:-2], date[-2:])

# Judge Reason
def getJudgeReason(txt):
    return txt.split('【裁判案由】')[1].split('【裁判全文】')[0]

# Judge Result
def getJudgeResult(txt):
    text = txt.split('主張')[0]
    results = {'win': True, 'lose': False}
    if '被告應給付原告' in text or '應給付原告' in text:
#         return results['win']
        return 'w'
    else:
#         return results['lose']
        return 'l'
    
# Paintiff Assertion
def getAssertionExisting(txt):
    condition = [False, False]
    conditions = [[1, 0], [0, 1], [1, 1], [0, 0]]
    if '兩造' in txt:
        condition = [True, True]
    if '原告主張' in txt or '原告起訴主張' in txt:
        condition[0] = True
    if '被告則以' in txt or '被告抗辯' in txt or '被告辯稱' in txt or '被告則辯稱' in txt or '答辯' in txt or '答辯聲明' in txt or '情詞置辯' in txt:
        condition[1] = True
#     return condition
    return str(condition).strip('[]')

# Accident Date
def getAccidentDate(txt):
    try:
        delimiter = u"，|。|、"
        ary = []
        for i in re.split(delimiter, ''.join(txt.decode('utf-8'))):
            if u'被告於民國' in i or u'分許' in i or u'時許' in i :
                ary.append(i)

        timeAry =[]
        term = ['民國']
        if len(ary) > 0:
            words = jieba.cut(ary[0], cut_all=False)
            for word in words:
                if word == term[0].decode('utf-8'):
                    yy = words.next()
                    words.next()
                    mm = words.next()
                    words.next()
                    dd = words.next()
                    if yy.isdigit() and mm.isdigit() and dd.isdigit():
                        timeAry = [int(yy), int(mm), int(dd)]
                    else:
                        timeAry = [yy, mm, dd]
                    break
#         return timeAry
        return str(timeAry).strip('[]')
    except:
#         return 'error'
        return None
    
# total damage
# TODO: deal with not only one sentence, and error, 參
def getTotalDamage(txt):
    try:
        section = txt.split('負擔')[0]
        if '原告之訴駁回' not in section and '均駁回' not in section and '上訴人' not in section:
            money = section.split('幣')[1].split('元')[0].decode('utf-8')
#             return [transform(money), money]
            return transform(money)
        else:
            return 0
    except:
#         return 'error'
        return None

In [16]:
# deal with fraction
def fractions(fraction):
    denominator = fraction.split(u'分之')[0]
    numerator = fraction.split(u'分之')[1]
    return float(transform(numerator))/float(transform(denominator))


#方法 transform-----輸入 money
def transform(money):
    money = ''.join(money.split(','))
    if money == u'百' or money == u'佰':
#         print 100
        return 100
    elif money == u'千' or money == u'仟':
#         print 1000
        return 1000
    elif money == u'十':
#         print 10
        return 10
    elif money.isdigit():
        return int(money)

    else:
        num=0      #for 中文數字的第幾個位置
        total = 0  #for 數字總合
        #------------
        get_m = [] #塞入money_unit

        get_n = [] #塞入number
        #------------
        simple_money_unit ={u'萬':10000,u'千':1000,u'百':100,u'十':10,u'仟':1000,u'佰':100,u'拾':10}
        simple_number={u'零':0,u'一':1,u'二':2,u'三':3,u'四':4,\
                       u'五':5,u'六':6,u'七':7,u'八':8,u'九':9}
        money_unit ={u'萬':10000,u'仟':1000,u'佰':100,u'拾':10,u'千':1000,u'百':100,u'十':10}
        number={u'零':0,u'壹':1,u'貳':2,u'參':3,u'肆':4,\
                u'伍':5,u'陸':6,u'柒':7,u'捌':8,u'玖':9}
        #------------
        #判斷是否為大寫數字 還小寫
        if money[0] in simple_number or money[0] in simple_money_unit:
            money_unit = simple_money_unit
            number = simple_number
#         print money #印出中文大寫數字


        for x in range(0,len(money)):
            if money[num]==u'零':#若是0就跳過
                num+=1
                continue
            elif money[num] in number:#若是 數字 開頭
                if num==len(money)-1:#判斷是否目前位置等於總長度    相等 是有個位數
                    get_n.append(number[money[num]])
                    get_m.append(1)
                    break

                get_n.append(number[money[num]])
                num+=1

                get_m.append(money_unit[money[num]])

                if num==len(money)-1:#若是沒有個位數 就直接結束
                    break
                num+=1

                if money[num] in money_unit:#只看是否是 萬  eg  壹拾萬  拾萬
                    get_n.append(0)
                    get_m.append(money_unit[money[num]])
                    if num==len(money)-1:
                        break
                    num+=1

                    if num==len(money)-1:#若是沒有個位數 就直接結束
                        break

            elif money[num] in money_unit:#若是 十 開頭  eg  拾萬
                get_n.append(1)
                get_m.append(money_unit[money[num]])
                num+=1

                if num==len(money)-1:
                    break

                if money[num] in money_unit:
                    #print num,0,money_unit[money[num]],'shit'
                    get_n.append(0)
                    get_m.append(money_unit[money[num]])
                    num+=1

                    if num==len(money)-1:
                        break

        #運用get_n get_m 倆倆一組 開始相乘
        times = len(get_n) if get_n>get_m else len(get_m)

        if 10000 in get_m:                 #若是有 "萬"
            change = get_m.index(10000)    #找出萬的位置
            for x in range(0,change):      #萬以前的單位 get_m 都乘 10000
                get_m[x] = get_m[x]*10000

        for x in range(0,times):           #get_m * get_n 在相加到 total
            total+=get_n[x]*get_m[x]

#         print total
        return total

import execjs
pyjs = execjs.compile("""    
    var range_77_13 = new Array(
        [100000, 1000], [1000000, 0.01], [10000000, 0.009], [100000000, 0.008], [1000000000, 0.007], [100000000000000, 0.006]
    );
    
    function calculate(fee) {
        var judprice = 0;

        var loc = fee.length;
        fee = rmoney(fee);
        var price = fee;
        if (price != "" && price != "0") {
            price = parseFloat(fee);
        }

        price2 = thou(price, 10000);
        judprice += cal77_13(thou(price, 10000), 1);
        judprice = thou2(judprice, 1);

        //form.judprice.value = fmoney(judprice);
        //form.judprice2.value = fmoney(Math.floor(judprice * 1.5));

        fee = fmoney(fee);
        //setCaretPosition(form.price, fee.length - loc);
        return fmoney(judprice);
    }


    function cal77_13(price, half) {
        var money = 0;
        for (i = 0; i < 6; i++) {
            if (i == 0) {
                money = multiplication(range_77_13[0][1], half);
                if (price <= range_77_13[0][0])
                    break;
            }

            if (i > 0) {
                if (price > range_77_13[i - 1][0] && price <= range_77_13[i][0]) {
                    money += multiplication(multiplication(multiplication((price - range_77_13[i - 1][0]), range_77_13[i][1]), half), 1.1);
                    break;
                } else {
                    money += multiplication(multiplication(multiplication((range_77_13[i][0] - range_77_13[i - 1][0]), range_77_13[i][1]), half), 1.1);
                }
            }
        }
        return money;
    }

    function thou(amin, nRound) {
        return parseInt((amin + (nRound - 1)) / nRound) * nRound;
    }

    function thou2(amin, nRound) {
        return parseInt(amin / nRound) * nRound;
    }

    function multiplication(num1, num2) {
        var m = 0,
            s1 = num1.toString(),
            s2 = num2.toString();
        try {
            m += s1.split(".")[1].length
        } catch (e) {}
        try {
            m += s2.split(".")[1].length
        } catch (e) {}
        return Number(s1.replace(".", "")) * Number(s2.replace(".", "")) / Math.pow(10, m)
    }

    function thou3(amin) {
        return Math.round(parseFloat(amin));
    }

    function countMoney() {
        calculate();
    }

    function lockNum() {
        if (event.keyCode == 13) return;
        if ((event.keyCode < 48) || (event.keyCode > 58)) {
            if (event.keyCode != 45)
                event.keyCode = 0;
        }
    }

    function doReset() {
        form.reset();
        countMoney();
    }

    function fmoney(s) {
        if (s == "" || s == "NaN") s = "0";
        n = 0;
        s = parseFloat((s + "").replace(/[^\d\.-]/g, "")).toFixed(n) + "";
        var l = s.split(".")[0].split("").reverse(),
            t = "";
        for (i = 0; i < l.length; i++) {
            t += l[i] + ((i + 1) % 3 == 0 && (i + 1) != l.length ? "" : "");
        }
        return t.split("").reverse().join("");
    }

    function rmoney(s) {
        if (s == "" || s == "NaN") s = "0";
        return s;
    }

    function setCaretPosition(ctrl, pos) {
        if (ctrl.setSelectionRange) {
            ctrl.focus();
            ctrl.setSelectionRange(pos, pos);
        } else if (ctrl.createTextRange) {
            var range = ctrl.createTextRange();
            range.collapse(true);
            range.moveEnd('character', pos);
            range.moveStart('character', pos);
            range.select();
        }
    }

    function doGetCaretPosition(ctrl) {
        var CaretPos = 0;
        // IE Support
        if (document.selection) {
            ctrl.focus();
            var Sel = document.selection.createRange();
            Sel.moveStart('character', -ctrl.value.length);
            CaretPos = Sel.text.length;
        }
        // Firefox support
        else if (ctrl.selectionStart || ctrl.selectionStart == '0')
            CaretPos = ctrl.selectionStart;
        return (CaretPos);
    }

    function setWidth() {
        tab.width = bdy.offsetWidth * 0.95;
    }
    
""")